In [66]:
import os
from dotenv import load_dotenv

load_dotenv()

NEO4J_URL = os.getenv("NEO4J_CONNECTION_URL")
NEO4J_USERNAME = os.getenv("NEO4J_USER")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [67]:
from typing import TypedDict, Annotated
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage

import operator

class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

In [68]:
from langgraph.checkpoint.sqlite import SqliteSaver

sqlite_memory = SqliteSaver.from_conn_string(":memory:")

In [104]:
from langgraph.graph import StateGraph, END

class AgentManager():

    def __init__(self, model, tools, system=""):
        self.system = system
        self.memory = sqlite_memory
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.__call)
        graph.set_entry_point("llm")
        graph.add_edge("llm", END)
        self.graph = graph.compile(checkpointer=self.memory)
        self.tools = {t.name: t for t in tools}
        self.model = model

        #self.model = model.bind_tools(tools)
        
    def __call(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}


In [94]:
from langchain.globals import set_verbose

set_verbose(True)

In [106]:
from langchain_openai import ChatOpenAI

prompt = "Você é um assitente amigável e deve responder de forma curta e direta."

model = ChatOpenAI(model="gpt-3.5-turbo")

agent_manager = AgentManager(model=model, tools=[], system=prompt)

In [109]:
import uuid

messages = [HumanMessage(content="ele nasceu e morreu em que ano?")]
config={"configurable": {"thread_id":123}}

for event in agent_manager.graph.stream(
    input={"messages": messages},
    config=config,
    stream_mode="values"
):
    event["messages"][-1].pretty_print()

# result = agent_manager.graph.invoke({"messages": messages}, config=config)

================================ Human Message =================================

ele nasceu e morreu em que ano?
================================== Ai Message ==================================

Albert Einstein nasceu em 1879 e faleceu em 1955.


In [100]:
print(result['messages'][-1].content)

Albert Einstein foi um físico teórico alemão que desenvolveu a teoria da relatividade e é conhecido por suas contribuições significativas para a física.


In [102]:
[message.pretty_print() for message in agent_manager.graph.checkpointer.list(config={"configurable": {"thread_id": 5}})]

[CheckpointTuple(config={'configurable': {'thread_id': '5', 'thread_ts': '1ef33b78-c234-60bf-8001-e228f227a18e'}}, checkpoint={'v': 1, 'ts': '2024-06-26T12:28:09.492703+00:00', 'id': '1ef33b78-c234-60bf-8001-e228f227a18e', 'channel_values': {'messages': [HumanMessage(content='quem foi Albert Einstein?'), AIMessage(content='Albert Einstein foi um físico teórico alemão que desenvolveu a teoria da relatividade e é conhecido por suas contribuições significativas para a física.', response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 37, 'total_tokens': 75}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-747feadf-79be-43a3-bfe9-e3005d874f88-0', usage_metadata={'input_tokens': 37, 'output_tokens': 38, 'total_tokens': 75})], 'llm': 'llm'}, 'channel_versions': {'__start__': 1, 'messages': 3, 'start:llm': 2, 'llm': 3}, 'versions_seen': {'__start__': {'__start__': 1}, 'llm': {'start:llm': 2}}, 'pending_sends': 